## Create Daily Feed Schedule

1. In general we limit the elephants to about 2 lbs of the goat pellets or biscuits per 24 hour period. For the deer feeders I would suggest a minimum of three feeds per 24 hour period and a maximum of 8 feeds per 24 hour period.

2. I would also suggest a minimum of one hour between intervals and a max of 8 hours hours between intervals. 

3. The following are times of the day I would not feed them for sleeping,  our routine daytime shifting, or during our keeper chat. 
    * 12am- 4am  sleeping break
    * 7:00am-10am  morning shifting and cleaning
    * 1:30pm-2:30pm keeper chat and afternoon feeding
    * 3:30pm- 6pm afternoon shifting and cleaning

Times to feed.
    * 5am- 7am
    * 10am-1:30pm
    * 2:30pm-3:30pm
    * 6pm- 12 am

In [1]:
import random
import time
from datetime import datetime, date, timedelta

In [2]:
today = date.today()
today = datetime(today.year, today.month, today.day, 0, 0, 0)
print("Today's date:", today)
tomorrow = today + timedelta(days=1)
tomorrow = datetime(tomorrow.year, tomorrow.month, tomorrow.day, 0, 0, 0)
print("Tomorrow's date:", tomorrow)

Today's date: 2020-01-01 00:00:00
Tomorrow's date: 2020-01-02 00:00:00


In [3]:
def randomtimes(stime, etime, n):
    td = etime - stime
    return [random.random() * td + stime for _ in range(n)]

def pidgeon_hole(n, n_bins): 
    quotient = n // n_bins
    remainder = n % n_bins

    bins = [quotient for i in range(n_bins)]    
    for i in range(remainder):
        bins[i] += 1
    return bins

In [4]:
feeders_dict = {
    0:[],
    1:[],
    2:[]
}

feeds = random.randint(3,8)

feeding_time_ranges = [(5,7),(10,13),(15,16),(18,24)]

bins = pidgeon_hole(feeds, len(feeding_time_ranges))

for feeding_time_range, b in zip(feeding_time_ranges, bins):
    start, end = feeding_time_range
    today = date.today()
    start_time = datetime(today.year, today.month, today.day, start, 0, 0)
    if end == 24:
        end_time = today + timedelta(days=1)
        end_time = datetime(end_time.year, end_time.month, end_time.day, 0, 0, 0)
    else:
        end_time = datetime(today.year, today.month, today.day, end, 0, 0)
    for time in randomtimes(start_time, end_time, b):
        feeders_dict[random.randint(0,2)].append(time)
        
print(feeders_dict)

{0: [datetime.datetime(2020, 1, 1, 12, 19, 30, 433877), datetime.datetime(2020, 1, 1, 15, 58, 0, 284880), datetime.datetime(2020, 1, 1, 19, 17, 3, 75153)], 1: [], 2: [datetime.datetime(2020, 1, 1, 5, 30, 16, 914590)]}


## Create Safe API Communication

- Utilize md5
- Figure out how to apply md5 to dictionary
- Research different methods of generating secret keys

In [ ]:
import hashlib
import json

def json_serial(obj):
    """JSON serializer for objects not serializable by default json code"""

    if isinstance(obj, (datetime, date)):
        return obj.isoformat()
    raise TypeError ("Type %s not serializable" % type(obj))

a = json.dumps(feeders_dict,default=json_serial, sort_keys = True).encode("utf-8")
hashlib.md5(a).hexdigest()

In [ ]:
from binascii import hexlify
import os

a+hexlify(os.urandom(4))

In [ ]:
# Server Side
serverTime = datetime.now().isoformat().encode("utf-8")

secret = 'hi, nice to meet you'.encode("utf-8")

password = hexlify(os.urandom(4))

digest = hashlib.md5(serverTime + secret).hexdigest().encode("utf-8")

In [ ]:
# Request
# requestTime=serverTime&digest=digest&requestPassword=md5(digest+password)
requestTime=serverTime
digest=digest
requestPassword = hashlib.md5(digest+password).hexdigest()

In [ ]:
# Client Side

# Calcuate based on inputs from request
requestDigest = hashlib.md5(requestTime + secret).hexdigest().encode("utf-8")

# If they're the same, the payload has not been altered
if digest == requestDigest:
# We can be assured that the request was initated by our server.

    passwordDigest = hashlib.md5(digest + password).hexdigest()
    # If the two matches, the password is valid
    if (requestPassword == passwordDigest):
        # YAY! Do something good.
        print("It worked")
    else:
        # Password doesn't match
        print("It doesn't work. requestPassword != passwordDigest")
        print(requestPassword, passwordDigest)

else:
    # Go away
    print("It doesn't work. digest != requestDigest")
    print(digest, requestDigest)